# Simple Example Agent - Usage Guide

This notebook demonstrates how to deploy and use the simple example agent from the MLRun marketplace.

## What This Agent Does

- **FastAPI-based service** with health check and echo endpoints
- **Consumes secrets**: API_KEY (required), DATABASE_PASSWORD (optional)
- **Consumes env vars**: SERVICE_NAME (required), LOG_LEVEL (optional), PORT (optional)
- **Demonstrates**: Complete marketplace agent lifecycle from packaging to deployment

## Step 1: Package the Agent

First, create a tar.gz archive of the agent code.

In [ ]:
import tarfile
import os

# Path to the agent directory
AGENT_DIR = "src"
ARCHIVE_NAME = "simple-example-agent.tar.gz"

# Create tar.gz archive
with tarfile.open(ARCHIVE_NAME, "w:gz") as tar:
    tar.add(AGENT_DIR, arcname=os.path.basename(AGENT_DIR))

print(f"✅ Created archive: {ARCHIVE_NAME}")
print(f"   Size: {os.path.getsize(ARCHIVE_NAME) / 1024:.2f} KB")

## Step 2: Load Agent Metadata

Load the agent metadata from `item.yaml`.

In [ ]:
import yaml

# Load metadata from item.yaml
with open("item.yaml", "r") as f:
    item_yaml = yaml.safe_load(f)

# Load requirements from requirements.txt
with open("requirements.txt", "r") as f:
    requirements = [line.strip() for line in f if line.strip() and not line.startswith("#")]

# Construct agent metadata for import
agent_metadata = {
    "name": item_yaml["name"],
    "version": item_yaml["version"],
    "author": item_yaml["author"],
    "description": item_yaml["description"],
    "kind": item_yaml["kind"],
    "protocol": item_yaml["agent_info"]["protocol"],
    "framework": item_yaml["agent_info"]["framework"],
    "asset_url": "",  # Will be set during deployment
    "requirements": requirements,
    "default_base_image": item_yaml["consumption_config"]["build"]["default_base_image"],
    "default_port": item_yaml["consumption_config"]["deploy"]["default_port"],
    "default_command": item_yaml["consumption_config"]["deploy"]["default_command"],
    "default_args": item_yaml["consumption_config"]["deploy"]["default_args"],
    "inputs": item_yaml["consumption_config"]["inputs"],
    "categories": item_yaml["categories"],
    "default_workdir": item_yaml["consumption_config"]["build"].get("default_workdir"),
    "build_extra": item_yaml["consumption_config"]["build"].get("build_extra", ""),
}

print("✅ Agent metadata loaded:")
print(f"   Name: {agent_metadata['name']}")
print(f"   Version: {agent_metadata['version']}")
print(f"   Framework: {agent_metadata['framework']}")
print(f"   Requirements: {len(requirements)} packages")

## Step 3: Upload Source Archive

Upload the agent archive to MLRun's artifact store.

In [ ]:
import mlrun

# Configuration
PROJECT_NAME = "agent-examples"

# Get or create project
project = mlrun.get_or_create_project(PROJECT_NAME, context="./")

# Upload source archive as artifact
source_artifact = project.log_artifact(
    "simple-agent-source",
    local_path=ARCHIVE_NAME
)

source_url = source_artifact.target_path
print(f"✅ Source uploaded to: {source_url}")

## Step 4: Import and Inspect Agent

Import the agent and view its information.

In [ ]:
# Import agent from marketplace
agent = mlrun.import_agent("simple-example-agent", agent_metadata)

# Display agent information
agent.info()

## Step 5: Deploy the Agent

Deploy the agent with required and optional configurations.

**Note**: Update the values below with your actual configuration.

In [ ]:
# Required configurations
API_KEY = "my-secret-api-key-12345"  # Required secret
SERVICE_NAME = "demo-echo-service"    # Required env var

# Optional configurations
DATABASE_PASSWORD = "my-db-password"  # Optional secret
LOG_LEVEL = "DEBUG"                   # Optional env var (default: INFO)

# Deploy the agent
url = agent.deploy(
    project=PROJECT_NAME,
    source_url=source_url,
    gateway_config={
        "name": "simple-agent-gateway",
        "authentication_mode": "none",  # No gateway-level auth (agent handles its own)
        "path": "/",
    },
    # Required inputs
    API_KEY=API_KEY,              # Secret (stored in K8s secret)
    SERVICE_NAME=SERVICE_NAME,    # Env var
    # Optional inputs
    DATABASE_PASSWORD=DATABASE_PASSWORD,  # Secret (optional)
    LOG_LEVEL=LOG_LEVEL,          # Env var (optional, default: INFO)
)

print(f"\n✅ Agent deployed successfully!")
print(f"URL: {url}")

## Step 6: Test the Deployed Agent

Test the agent endpoints.

In [ ]:
import requests
import urllib3

# Disable SSL warnings for self-signed certs
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Construct full URL
if not url.startswith("http"):
    test_url = f"https://{url}"
else:
    test_url = url

print(f"Testing agent at: {test_url}\n")

### Test 1: Health Check

In [ ]:
# Test health endpoint
response = requests.get(f"{test_url}/health", verify=False)
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

### Test 2: Root Endpoint

In [ ]:
# Test root endpoint
response = requests.get(test_url, verify=False)
print(f"Status: {response.status_code}")
print(f"Response: {response.json()}")

### Test 3: Echo Endpoint (Valid API Key)

In [ ]:
# Test echo endpoint with valid API key
payload = {
    "message": "Hello from MLRun!",
    "uppercase": False
}

headers = {
    "X-API-Key": API_KEY  # Use the API key we configured
}

response = requests.post(
    f"{test_url}/echo",
    json=payload,
    headers=headers,
    verify=False
)

print(f"Status: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"\n✅ Echo Response:")
    print(f"   Service: {result['service_name']}")
    print(f"   Original: {result['original_message']}")
    print(f"   Processed: {result['processed_message']}")
    print(f"   Has Database: {result['has_database']}")
else:
    print(f"Error: {response.text}")

### Test 4: Echo Endpoint (Invalid API Key)

In [ ]:
# Test echo endpoint with invalid API key (should fail)
payload = {
    "message": "This should fail",
    "uppercase": False
}

headers = {
    "X-API-Key": "wrong-api-key"
}

response = requests.post(
    f"{test_url}/echo",
    json=payload,
    headers=headers,
    verify=False
)

print(f"Status: {response.status_code}")
if response.status_code == 401:
    print("✅ Correctly rejected invalid API key")
else:
    print(f"Unexpected response: {response.text}")

### Test 5: Echo Endpoint with Uppercase

In [ ]:
# Test echo endpoint with uppercase option
payload = {
    "message": "Convert me to uppercase!",
    "uppercase": True
}

headers = {
    "X-API-Key": API_KEY
}

response = requests.post(
    f"{test_url}/echo",
    json=payload,
    headers=headers,
    verify=False
)

print(f"Status: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(f"\n✅ Uppercase Echo Response:")
    print(f"   Original: {result['original_message']}")
    print(f"   Processed: {result['processed_message']}")
else:
    print(f"Error: {response.text}")

## Step 7: Test Redeployment (Cache Optimization)

Redeploy the agent - should be much faster as it reuses the cached base image.

In [ ]:
import time

# Measure redeployment time
start_time = time.time()

url2 = agent.deploy(
    project=PROJECT_NAME,
    source_url=source_url,
    gateway_config={
        "name": "simple-agent-gateway",
        "authentication_mode": "none",
    },
    API_KEY=API_KEY,
    SERVICE_NAME=SERVICE_NAME,
    DATABASE_PASSWORD=DATABASE_PASSWORD,
    LOG_LEVEL=LOG_LEVEL,
)

elapsed_time = time.time() - start_time

print(f"\n✅ Agent redeployed successfully!")
print(f"   Time: {elapsed_time:.1f} seconds (cached image used)")
print(f"   URL: {url2}")

## Step 8: Force Rebuild (Optional)

Force a complete rebuild by setting `force_rebuild=True`.

In [ ]:
# Force rebuild (will take longer)
# Uncomment to test:

# url3 = agent.deploy(
#     project=PROJECT_NAME,
#     source_url=source_url,
#     force_rebuild=True,  # Force rebuild
#     gateway_config={
#         "name": "simple-agent-gateway",
#         "authentication_mode": "none",
#     },
#     API_KEY=API_KEY,
#     SERVICE_NAME=SERVICE_NAME,
#     DATABASE_PASSWORD=DATABASE_PASSWORD,
#     LOG_LEVEL=LOG_LEVEL,
# )

# print(f"\n✅ Agent rebuilt and deployed!")
# print(f"   URL: {url3}")

print("⚠️ Force rebuild commented out to save time. Uncomment to test.")

## Summary

This notebook demonstrated:

1. ✅ **Packaging** - Created tar.gz archive of agent code
2. ✅ **Metadata** - Loaded agent metadata from item.yaml
3. ✅ **Upload** - Uploaded source to MLRun artifact store
4. ✅ **Import** - Imported agent using marketplace API
5. ✅ **Deploy** - Deployed agent with secrets and env vars
6. ✅ **Test** - Tested all endpoints (health, echo, auth)
7. ✅ **Cache** - Demonstrated fast redeployment using cached images

### Key Takeaways

- **Secrets** (type=secret): Stored in K8s secrets, available as env vars
- **Env vars** (type=env): Set directly as environment variables
- **Required inputs**: Must be provided during deployment
- **Optional inputs**: Use defaults from item.yaml if not provided
- **Caching**: Redeployment reuses built images for faster deployment
- **Force rebuild**: Use `force_rebuild=True` to rebuild from scratch

### Next Steps

- Customize the agent code in `simple_agent/main.py`
- Update metadata in `item.yaml`
- Add more dependencies to `requirements.txt`
- Deploy your customized agent!